In [0]:

%sql
USE CATALOG trabalho_etl_bd3;
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;


In [0]:
%sql
SELECT
  data_pedido,
  try_to_timestamp(data_pedido, 'yyyy-MM-dd HH:mm') AS convertido
FROM trabalho_etl_bd3.silver.pedidos
LIMIT 20;


data_pedido,convertido
2025-06-09 17:10,2025-06-09T17:10:00.000Z
2025-06-04 09:13,2025-06-04T09:13:00.000Z
2025-09-27 18:48,2025-09-27T18:48:00.000Z
2025-07-21 06:37,2025-07-21T06:37:00.000Z
2025-06-25 11:10,2025-06-25T11:10:00.000Z
2025-09-11 22:15,2025-09-11T22:15:00.000Z
2025-09-10 00:43,2025-09-10T00:43:00.000Z
2025-09-05 12:54,2025-09-05T12:54:00.000Z
2025-08-25 03:52,2025-08-25T03:52:00.000Z
2025-08-08 03:57,2025-08-08T03:57:00.000Z


Tabela Vendas por Mês

In [0]:
%sql
CREATE OR REPLACE TABLE gold.vendas_por_mes AS
WITH pedidos_norm AS (
  SELECT
    try_to_timestamp(data_pedido, 'yyyy-MM-dd HH:mm') AS data_pedido_ts,
    valor_total,
    id_pedido,
    try_cast(nota_avaliacao AS DOUBLE) AS nota_avaliacao_num
  FROM trabalho_etl_bd3.silver.pedidos
)
SELECT
  date_format(data_pedido_ts, 'yyyy-MM') AS ano_mes,
  SUM(valor_total)                        AS valor_total,
  COUNT(DISTINCT id_pedido)              AS qtd_pedidos,
  AVG(valor_total)                        AS ticket_medio,
  AVG(nota_avaliacao_num)                 AS nota_media
FROM pedidos_norm
WHERE data_pedido_ts IS NOT NULL
GROUP BY 1
ORDER BY 1;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM gold.vendas_por_mes ORDER BY ano_mes;


ano_mes,valor_total,qtd_pedidos,ticket_medio,nota_media
2025-05,1466.35,11,133.30454545454543,3.75
2025-06,2373.1299999999997,24,98.88041666666665,4.388888888888889
2025-07,1863.89,21,88.75666666666667,4.375
2025-08,2539.6099999999997,25,101.58439999999999,4.380952380952381
2025-09,3684.39,32,111.64818181818181,4.066666666666666


Entregas por Estado

In [0]:
CREATE OR REPLACE TABLE gold.entregas_por_estado AS
WITH pedidos_norm AS (
  SELECT id_pedido, TRY_CAST(regexp_extract(CAST(id_cliente AS STRING), '(\\d+)', 1) AS INT) AS id_cliente_norm
  FROM trabalho_etl_bd3.silver.pedidos
),
clientes_norm AS (
  SELECT TRY_CAST(regexp_extract(CAST(id_cliente AS STRING), '(\\d+)', 1) AS INT) AS id_cliente_norm, uf
  FROM trabalho_etl_bd3.silver.clientes
)
SELECT
  c.uf,
  COUNT(DISTINCT e.id_pedido) AS qtd_entregas,
  ROUND(AVG(e.atraso_convertido) * 100, 2) AS taxa_atraso_pct
FROM (
  SELECT id_pedido,
         CASE WHEN atrasado_flag='teve atraso' THEN 1.0
              WHEN atrasado_flag='não teve atraso' THEN 0.0 END AS atraso_convertido
  FROM trabalho_etl_bd3.silver.entregas
) e
LEFT JOIN pedidos_norm  p ON e.id_pedido = p.id_pedido
LEFT JOIN clientes_norm c ON p.id_cliente_norm = c.id_cliente_norm
WHERE c.uf IS NOT NULL                      
GROUP BY c.uf
ORDER BY c.uf;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
  uf,
  qtd_entregas,
  ROUND(taxa_atraso_pct, 2) AS taxa_atraso_pct -- agora sim em porcentagem correta
FROM gold.entregas_por_estado
ORDER BY uf;


uf,qtd_entregas,taxa_atraso_pct
AL,1,50.00
AM,3,20.00
DF,5,50.00
ES,3,66.67
GO,5,70.00
MA,6,83.33
MS,5,71.43
MT,2,100.00
PA,1,100.00
PB,4,83.33


Ranking Clientes

In [0]:
%sql
CREATE OR REPLACE TABLE gold.ranking_clientes AS
WITH pedidos_norm AS (
  SELECT
    id_pedido,
    CAST(id_cliente AS INT) AS id_cliente_norm,
    valor_total,
    try_to_timestamp(data_pedido, 'yyyy-MM-dd HH:mm') AS data_pedido_ts
  FROM trabalho_etl_bd3.silver.pedidos
),
clientes_norm AS (
  SELECT
    CAST(regexp_extract(CAST(id_cliente AS STRING), '(\\d+)', 1) AS INT) AS id_cliente_norm,
    nome,
    uf,
    cidade
  FROM trabalho_etl_bd3.silver.clientes
)
SELECT
  RANK() OVER (ORDER BY SUM(p.valor_total) DESC) AS posicao,
  c.nome,
  c.uf,
  c.cidade,
  ROUND(SUM(p.valor_total), 2) AS total_gasto,
  COUNT(p.id_pedido) AS qtd_pedidos,
  ROUND(AVG(p.valor_total), 2) AS ticket_medio,
  MAX(p.data_pedido_ts) AS ultima_compra
FROM pedidos_norm p
LEFT JOIN clientes_norm c
  ON p.id_cliente_norm = c.id_cliente_norm
GROUP BY c.nome, c.uf, c.cidade
ORDER BY posicao;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.ranking_clientes
ORDER BY posicao
LIMIT 20;


posicao,nome,uf,cidade,total_gasto,qtd_pedidos,ticket_medio,ultima_compra
1,Bruna Aragão,ES,Duarte,645.98,5,129.2,2025-09-12T23:58:00.000Z
2,Alexia Nunes,GO,da Cunha da Prata,595.99,5,119.2,2025-08-25T03:52:00.000Z
3,Otávio Silva,PI,Vieira,489.95,4,122.49,2025-09-28T12:37:00.000Z
4,Sabrina Souza,AM,Rocha da Mata,488.09,3,162.7,2025-09-11T22:15:00.000Z
5,Augusto Santos,DF,Viana do Amparo,461.12,5,92.22,2025-09-05T12:54:00.000Z
6,Maria Eduarda Pereira,RN,da Cruz do Campo,447.79,5,89.56,2025-09-13T10:33:00.000Z
7,Ana Lívia Pinto,SP,Rezende de Gomes,439.51,3,146.5,2025-09-06T06:52:00.000Z
8,Vitor Azevedo,RS,Teixeira,438.48,5,87.7,2025-08-17T11:21:00.000Z
9,Manuela Gonçalves,DF,Silveira de Costela,376.77,4,94.19,2025-09-23T20:04:00.000Z
10,Ana Laura Castro,TO,da Cruz Paulista,370.31,3,123.44,2025-09-23T19:47:00.000Z


In [0]:
%sql
SHOW TABLES IN trabalho_etl_bd3.gold;



database,tableName,isTemporary
gold,entregas_por_estado,false
gold,ranking_clientes,false
gold,vendas_por_mes,false


In [0]:
CREATE VOLUME trabalho_etl_bd3.gold.gold_output;


In [0]:
SHOW VOLUMES IN trabalho_etl_bd3.gold;


database,volume_name
gold,gold_output


In [0]:
%python
df = spark.table("gold.entregas_por_estado")

df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Volumes/trabalho_etl_bd3/gold/gold_output/entregas_por_estado_csv")


In [0]:
%python
df = spark.table("gold.ranking_clientes")

df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Volumes/trabalho_etl_bd3/gold/gold_output/ranking_clientes_csv")



In [0]:
%python
df = spark.table("gold.vendas_por_mes")

df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Volumes/trabalho_etl_bd3/gold/gold_output/vendas_por_mes_csv")
